# ToDo
1. ロジスティック回帰を用いることで過去手法と比較
    - 係数をそのまま使用
    - 特徴量をそのまま使用
    - 今回の特徴量で
1. BMIのグルーピングを最重量のグループかどうかでグルーピング
1. 車両の面積を特徴量として入れてみる
1. 決定木の可視化
1. 親ノードを調整
1. ベイジアンネットワークの勉強
1. GNNの調査  
不要かも
1. PLSA  
不要かも

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import sys
import seaborn as sns
import xgboost as xgb
import imblearn
import tqdm
import graphviz
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, BorderlineSMOTE, SMOTENC
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, ClusterCentroids, NearMiss
from imblearn.combine import SMOTETomek, SMOTEENN
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import  confusion_matrix, classification_report, roc_curve, roc_auc_score, auc, accuracy_score, precision_recall_curve, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.compose import ColumnTransformer
from IPython import embed
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import HTML
from dtreeviz.trees import dtreeviz
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 200)
pd.options.display.float_format = '{:.4g}'.format
class pycolor:
    BLACK = '\033[30m'
    RED = '\033[31m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    BLUE = '\033[34m'
    PURPLE = '\033[35m'
    CYAN = '\033[36m'
    WHITE = '\033[37m'
    END = '\033[0m'
    BOLD = '\038[1m'
    UNDERLINE = '\033[4m'
    INVISIBLE = '\033[08m'
    REVERCE = '\033[07m'

## グルーピング用関数

In [ ]:
def grouping(df):
    group_list = [['MODELYR', 'MODELG', [1990, 1997, 2001]],
                  ['CURBWGT', 'WGTG', [101, 201]], 
                  ['otvehwgt', 'owgtg', [101, 201]],
                  ['AGE', 'AGEG', [60, 70]],
                  ['BMI', 'BMIGJP', [18.5, 25, 30, 35, 40]],
                  ['PDOF1', 'PDOFG', [35, 145, 215, 325]]
                 ]
    for glist in group_list:
        if glist[0] in list(df.columns):
            for i in range(len(glist[2]) + 1):
                if i == 0:
                    df.loc[df[glist[0]] < glist[2][0], glist[1]] = 0
                elif i < len(glist[2]):
                    df.loc[(df[glist[0]] >= glist[2][i - 1]) & (df[glist[0]] < glist[2][i]), glist[1]] = i
                else:
                    df.loc[df[glist[0]] >= glist[2][-1], glist[1]] = i
    df.loc[df['PDOFG'] == 4, 'PDOFG'] = 0
    #df['POS_DOF'] = 0
    #df.loc[(((df['PDOF1'] >= 0) & (df['PDOF1'] <= 110)) | ((df['PDOF1'] >= 340) & (df['PDOF1'] <= 360))) & ((df['SEATLR2'] == 2) & (df['SEATROW2'] == 1)), 'POS_DOF'] = 1
    #df.loc[((df['PDOF1'] >= 70) & (df['PDOF1'] <= 200)) & ((df['SEATLR2'] == 2) & (df['SEATROW2'] == 2)), 'POS_DOF'] = 1
    #df.loc[((df['PDOF1'] >= 160) & (df['PDOF1'] <= 290)) & ((df['SEATLR2'] == 1) & (df['SEATROW2'] == 2)), 'POS_DOF'] = 1
    #df.loc[(((df['PDOF1'] >= 250) & (df['PDOF1'] <= 360)) | ((df['PDOF1'] >= 0) & (df['PDOF1'] <= 20))) & ((df['SEATLR2'] == 1) & (df['SEATROW2'] == 1)), 'POS_DOF'] = 1    
    for i in ['MODELYR', 'CURBWGT', 'otvehwgt', 'AGE', 'BMI', 'PDOF1']:
        if i in df.columns:
            df = df.drop(columns = i, axis = 1)
    return df

## データの読み込み

In [ ]:
path = os.path.dirname(os.path.abspath('__file__'))
df = pd.read_csv(os.path.join(path, 'grouped_data.csv'))

## データに含まれるカラム

In [ ]:
df.columns

## リサンプル用数値振り直し

In [ ]:
map_dic = {'MANCOLL': {0:4, 1:5, 2:3, 4:1, 5:0, 6:2},
           'BODYG': {0:4, 1:1, 2:2, 3:3, 4:0},
           'obodyg': {0:0, 1:1, 2:3, 3:2, 4:4, 5:5}}
for i in map_dic.keys():
    #count = df[i].isnull().sum()
    df[i] = df[i].map(map_dic[i])
    #df[i].isnull().sum() - count

## 不要なカラムをドロップ

In [ ]:
#上下どちらかのセルを選択して実行
df = df.drop(columns = 'DVper10')

In [ ]:
df = df.drop(columns = 'DVTOTAL')

In [ ]:
df = df.drop(columns = 'TRAVELSP')

## NANと同じ意味のデータをNANに置換

In [ ]:
df['TRAVELSP'] = df['TRAVELSP'].replace(777, np.nan)

## 相関係数の高い特徴量をドロップ

In [ ]:
#全てグルーピング後相関係数に応じて特徴量を削除したデータを作成
df_dic  = {}
count = 0
for i, col1 in enumerate([['MODELG', 'SEATROW2'], ['BAGAVAIL', 'BAGAVOTH']]):
    for j, col2 in enumerate([['BODYG', 'obodyg'], ['WGTG',  'owgtg']]):
        df_dic[count] = df.drop(columns = col1 + col2)
        #df_dic[count] = df.drop(columns = 'POS_DOF')
        print(count)
        print('remain : ', list(df_dic[count].columns))
        print('drop : ', col1 + col2)
        count += 1
df_dic[count] = df

In [ ]:
#連続量の場合に相関係数に応じて特徴量を削除
df_dic  = {}
count = 0
for i, col1 in enumerate([['MODELYR', 'SEATROW2'], ['BAGAVAIL', 'BAGAVOTH']]):
    for j, col2 in enumerate([['BODYG', 'obodyg'], ['CURBWGT',  'otvehwgt']]):
        df_dic[count] = df.drop(columns = col1 + col2)
        #df_dic[count] = df.drop(columns = 'POS_DOF')
        print(count)
        print('remain : ', list(df_dic[count].columns))
        print('drop : ', col1 + col2)
        count += 1
df_dic[count] = df

## グルーピング
リサンプリングしない場合はここでグルーピングを行う

In [ ]:
for i in range(2, 5):
    df_dic[i] = grouping(df_dic[i])

## 標準化
ランダムフォレストのみなら不要

In [ ]:
stdcol = ['SPLIMIT', 'TRAVELSP', 'DVTOTAL']
for i in range(5):
    for j in stdcol:
        std = StandardScaler()
        df_std = df_dic[i].query('YEAR < 2015')
        if j in df_std.columns:
            null_val = df_std[j].isnull()
            std.fit(df_std.loc[~null_val, [j]])
            null_val = df_dic[i][j].isnull()
            df_dic[i].loc[~null_val, [j]] = std.transform(df_dic[i].loc[~null_val, [j]])

## 中央値で欠損値補完

In [ ]:
for i in range(5):
    for j in list(df_dic[i].columns):
        df_med = df_dic[i].query('YEAR < 2015')
        med = df_med[j].median()
        if df_dic[i][j].isnull().any():
            print(j, ':', med)
        df_dic[i][j] = df_dic[i][j].fillna(med)

## 欠損値補完の確認

In [ ]:
for i in range(5):
    print('欠損値数：', df_dic[i].isnull().any().sum())

## テストデータと訓練データの分割

In [ ]:
train = {}
test = {}
explanatory = {}
y_train = df.query('YEAR < 2015')['MAIS']
y_train3 = df.query('YEAR < 2015')['MAIS3']
y_test = df.query('YEAR == 2015')['MAIS']
y_test3 = df.query('YEAR == 2015')['MAIS3']
objective = df['MAIS']
objective3 = df['MAIS3']
for i in range(5):
    train[i] = df_dic[i].query('YEAR < 2015').drop(columns = ['YEAR', 'MAIS', 'MAIS3'])
    test[i] = df_dic[i].query('YEAR == 2015').drop(columns = ['YEAR', 'MAIS', 'MAIS3'])
    explanatory[i] = df_dic[i].drop(columns = ['YEAR', 'MAIS', 'MAIS3'])

## ダミー変数化

In [ ]:
dummy_dic = {}
not_dummy = ['YEAR', 'MAIS', 'MAIS3', 'DVTOTAL', 'TRAVELSP']
for i in range(5):
    dummy_dic[i] = pd.get_dummies(df_dic[i], drop_first = False, columns = [x for x in list(df_dic[i].columns) if x not in not_dummy])
#ダミー変数化したデータをテストデータと訓練データに分割
train_dummy =  {}
test_dummy =  {}
for i in range(5):
    train_dummy[i] = dummy_dic[i].query('YEAR < 2015').drop(columns = ['YEAR', 'MAIS', 'MAIS3'])
    test_dummy[i] = dummy_dic[i].query('YEAR == 2015').drop(columns = ['YEAR', 'MAIS', 'MAIS3'])

## ランダムフォレストで推定

In [ ]:
forest_dic = {}
gs_result = {}
predict = {}#テストデータセットの推定結果
pred = {}#訓練データでの推定結果
f_imp_dic = {}
grid_param = {'max_depth': [5], 'min_samples_split': [2], 'min_samples_leaf': [2], 'n_estimators': [500]}
for i in range(5):
    forest_dic[i] = GridSearchCV(RandomForestClassifier(random_state = 0, class_weight = 'balanced'), grid_param, cv = 5, scoring = 'roc_auc')
    forest_dic[i].fit(train[i], y_train3)
    predict[i] = forest_dic[i].predict(test[i])
    fpr, tpr, thresholds = roc_curve(y_test3, predict[i], pos_label = 1)
    print('best parameter:', forest_dic[i].best_params_)
    print('dic num:', i)
    print('test auc:', auc(fpr, tpr))
    print('test accuracy:', accuracy_score(y_test3, predict[i]) * 100)
    pred[i] = forest_dic[i].predict(train[i])
    fpr, tpr, thresholds = roc_curve(y_train3, pred[i], pos_label = 1)
    print('train auc:', auc(fpr, tpr))
    print('train accuracy:', accuracy_score(y_train3, pred[i]) * 100)
    gs_result[i] = pd.DataFrame.from_dict(forest_dic[i].cv_results_)
    f_imp_dic[i] = pd.DataFrame(index = range(len(train[i].columns)))
    imp = forest_dic[i].best_estimator_.feature_importances_
    for j, col in enumerate(train[i].columns):
        f_imp_dic[i].at[j, 'column'] = col
        f_imp_dic[i].at[j, 'importance'] = imp[j]
    f_imp_dic[i] = f_imp_dic[i].sort_values('importance', ascending = False)

In [ ]:
for i in range(5):
    f_imp_dic[i].head(10)

## 実際のMAIS3+が1で推定結果が0となっているデータの出力

In [ ]:
hoge = df.query('YEAR == 2015')
hoge['pre'] = predict[4]
hoge.drop(columns = ['YEAR'], inplace = True)
len(hoge.query('MAIS3 == 1'))
len(hoge.query('MAIS3 == 1 & pre == 0'))
print('真値1, 推定値0のデータ')
hoge.query('MAIS3 == 1 & pre == 0')
print('真値1, 推定値1のデータ')
hoge.query('MAIS3 == 1 & pre == 1').sort_values('MAIS', ascending = False)

## ロジスティック回帰で推定

In [ ]:
logi_dic = {}
coef_df = {}
grid_param = {'C': [0.1, 0.5, 1], 'solver': ['liblinear']}
for i in range(5):
    logi_dic[i] = GridSearchCV(LogisticRegression(random_state = 0, class_weight = 'balanced'), grid_param, cv = 5, scoring = 'roc_auc')
    logi_dic[i].fit(train_dummy[i], y_train3)
    predict = logi_dic[i].predict(test_dummy[i])
    fpr, tpr, thresholds = roc_curve(y_test3, predict, pos_label = 1)
    coef_df[i] = pd.DataFrame({'Feature' : list(train_dummy[i].columns.values), 'coef' : logi_dic[i].best_estimator_.coef_[0]})
    coef_df[i]['abs'] = coef_df[i]['coef'].abs()
    coef_df[i] = coef_df[i].sort_values('abs', ascending = False).drop('abs', axis = 1)
    print('best parameter:', logi_dic[i].best_params_)
    print('dic num:', i)
    print('test auc: ', auc(fpr, tpr))    
    predict = logi_dic[i].predict(train_dummy[i])
    fpr, tpr, thresholds = roc_curve(y_train3, predict, pos_label = 1)
    print('train auc: ', auc(fpr, tpr))

In [ ]:
for i in range(5):
    coef_df[1].head(10)

## リサンプリング(標準化前)

In [ ]:
sampling_train = {}
hoge = {}
ncl = [[0, 2, 3, 4], [0, 2, 3, 4, 5, 6], [0, 2, 3, 4, 5, 6, 7, 8]]
for i in range(5):
    trainsample = df_dic[i].query('YEAR < 2015')
    trainy = trainsample['MAIS3']
    trainX = trainsample.drop(columns = ['YEAR', 'MAIS', 'MAIS3'])
    #print('ros')
    ros = RandomOverSampler(random_state = 0)
    sampling_train['ros_X_{}'.format(i)], sampling_train['ros_y_{}'.format(i)] = ros.fit_sample(trainX, trainy)
    #print('rus')
    rus = RandomUnderSampler(random_state = 0)
    sampling_train['rus_X_{}'.format(i)], sampling_train['rus_y_{}'.format(i)] = rus.fit_sample(trainX, trainy)
    #print('tl')
    tl = TomekLinks(random_state = 0, ratio = 'majority')
    temp_df_X, temp_df_y = tl.fit_sample(trainX, trainy)
    sampling_train['tl_X_{}'.format(i)] = pd.DataFrame(temp_df_X, columns = trainX.columns)
    sampling_train['tl_y_{}'.format(i)] = pd.DataFrame(temp_df_y, columns = ['MAIS3'])
    #print('cc')
    cc = ClusterCentroids(random_state = 0, ratio = 'majority')
    temp_df_X, sampling_train['cc_y_{}'.format(i)] = cc.fit_sample(trainX, trainy)
    sampling_train['cc_X_{}'.format(i)] = pd.DataFrame(temp_df_X, columns = trainX.columns).round()
    #print('smote')
    smote = SMOTE(random_state = 0, ratio = 'minority')
    temp_df_X,sampling_train['smote_y_{}'.format(i)] = smote.fit_sample(trainX, trainy)
    sampling_train['smote_X_{}'.format(i)] = pd.DataFrame(temp_df_X, columns = trainX.columns).round() 
    #print('smt')
    smt = SMOTETomek(random_state = 0, ratio = 'auto')
    temp_df_X, sampling_train['smt_y_{}'.format(i)] = smt.fit_sample(trainX, trainy)
    sampling_train['smt_X_{}'.format(i)] = pd.DataFrame(temp_df_X, columns = trainX.columns).round()
    #print('bsmote')
    bsmote = BorderlineSMOTE(random_state = 0)
    temp_df_X,sampling_train['bsmote_y_{}'.format(i)] = bsmote.fit_sample(trainX, trainy)
    sampling_train['bsmote_X_{}'.format(i)] = pd.DataFrame(temp_df_X, columns = trainX.columns).round() 
    #print('adasyn')
    adasyn = ADASYN(random_state = 0)
    temp_df_X,sampling_train['adasyn_y_{}'.format(i)] = adasyn.fit_sample(trainX, trainy)
    sampling_train['adasyn_X_{}'.format(i)] = pd.DataFrame(temp_df_X, columns = trainX.columns).round()
    #print('smotenc')
    #ncl = list(range(len(df_dic[i].columns) - 3))#事前にグルーピング
    #ncl.pop(1)
    #ncl.pop(-1)
    #smotenc = SMOTENC(categorical_features = ncl, random_state = 0)
    #temp_df_X,sampling_train['smotenc_y_{}'.format(i)] = smotenc.fit_sample(trainX, trainy)
    #sampling_train['smotenc_X_{}'.format(i)] = pd.DataFrame(temp_df_X, columns = trainX.columns)
for i in range(5):
    for j in ['ros', 'rus']:
        sampling_train['{}_X_{}'.format(j, i)] = pd.DataFrame(sampling_train['{}_X_{}'.format(j, i)], columns = df_dic[i].drop(columns = ['YEAR', 'MAIS', 'MAIS3']).columns)

## Tomeklinks + Oversampling

In [ ]:
tl_over = {}
for i in range(5):
    trainy = sampling_train['tl_y_{}'.format(i)]['MAIS3']
    trainX = sampling_train['tl_X_{}'.format(i)]
    #print('ros')
    ros = RandomOverSampler(random_state = 0)
    temp_df_X, tl_over['ros_y_{}'.format(i)] = ros.fit_sample(trainX, trainy)
    tl_over['ros_X_{}'.format(i)] = pd.DataFrame(temp_df_X, columns = trainX.columns)
    #print('bsmote')
    bsmote = BorderlineSMOTE(random_state = 0)
    temp_df_X,tl_over['bsmote_y_{}'.format(i)] = bsmote.fit_sample(trainX, trainy)
    tl_over['bsmote_X_{}'.format(i)] = pd.DataFrame(temp_df_X, columns = trainX.columns).round() 
    #print('adasyn')
    adasyn = ADASYN(random_state = 0)
    temp_df_X,tl_over['adasyn_y_{}'.format(i)] = adasyn.fit_sample(trainX, trainy)
    tl_over['adasyn_X_{}'.format(i)] = pd.DataFrame(temp_df_X, columns = trainX.columns).round()
    #print('smotenc')
    #ncl = list(range(len(df_dic[i].columns) - 3))#事前にグルーピング
    #ncl.pop(1)
    #ncl.pop(-1)
    #smotenc = SMOTENC(categorical_features = ncl, random_state = 0)
    #temp_df_X,tl_over['smotenc_y_{}'.format(i)] = smotenc.fit_sample(trainX, trainy)
    #tl_over['smotenc_X_{}'.format(i)] = pd.DataFrame(temp_df_X, columns = trainX.columns)

## リサンプリング後グルーピング

In [ ]:
for i in range(3, 5):
    for j in ['ros', 'rus', 'tl', 'cc', 'smote', 'smt', 'bsmote', 'adasyn']:#, 'smotenc']:
        sampling_train['{}_X_{}'.format(j, i)] = grouping(sampling_train['{}_X_{}'.format(j, i)])

In [ ]:
for i in range(3, 5):
    for j in ['ros', 'bsmote', 'adasyn']:#, 'smotenc']:
        tl_over['{}_X_{}'.format(j, i)] = grouping(tl_over['{}_X_{}'.format(j, i)])

In [ ]:
for i in range(2, 5):
    for j in ['ros', 'rus', 'tl', 'cc', 'smote', 'smt', 'bsmote', 'adasyn', 'smotenc']:
        print(i, j)
        np.count_nonzero(sampling_train['{}_y_{}'.format(j, i)] == 0) - np.count_nonzero(sampling_train['{}_y_{}'.format(j, i)] == 1)

## テストデータをグルーピング

In [ ]:
for i in range(2, 5):
    test[i] = grouping(test[i])

## リサンプリングした訓練データを用いてランダムフォレストで推定

In [ ]:
resample_forest = pd.DataFrame(index = range(45),columns = ['サンプリング方法', 'データ番号', 'test auc', 'test accuracy', 'train auc', 'train accuracy', 'auc diff', 'best param']) 
f_imp_dic = {}
count = 0
grid_param = {'max_depth': [5], 'min_samples_split': [2], 'min_samples_leaf': [2], 'n_estimators': [500]}
for j in ['ros', 'rus', 'tl', 'smote', 'smt', 'bsmote', 'adasyn', 'cc']:#, 'smotenc']:
    for i in range(5):
        forest = GridSearchCV(RandomForestClassifier(random_state = 0, class_weight = 'balanced'), grid_param, cv = 5, scoring = 'roc_auc') 
        if j == 'tl':
            forest = GridSearchCV(RandomForestClassifier(random_state = 0, class_weight = 'balanced'), grid_param, cv = 5, scoring = 'roc_auc') 
        forest.fit(sampling_train['{}_X_{}'.format(j, i)], sampling_train['{}_y_{}'.format(j, i)])
        pred = forest.best_estimator_.predict(test[i])
        fpr, tpr, thresholds = roc_curve(y_test3, pred, pos_label = 1)
        test_auc = auc(fpr, tpr)
        resample_forest['サンプリング方法'][count] = j
        resample_forest['データ番号'][count] = i
        resample_forest['best param'][count] = forest.best_params_
        resample_forest['test auc'][count] = test_auc
        resample_forest['test accuracy'][count] = accuracy_score(y_test3, pred) * 100
        pred = forest.best_estimator_.predict(sampling_train['{}_X_{}'.format(j, i)])
        fpr, tpr, thresholds = roc_curve(sampling_train['{}_y_{}'.format(j, i)], pred, pos_label = 1)
        train_auc = auc(fpr, tpr)
        resample_forest['train auc'][count] = train_auc
        resample_forest['train accuracy'][count] = accuracy_score(sampling_train['{}_y_{}'.format(j, i)], pred) * 100
        resample_forest['auc diff'][count] = train_auc - test_auc
        f_imp_dic['{}_{}'.format(j, i)] = pd.DataFrame(index = range(len(sampling_train['{}_X_{}'.format(j, i)].columns)))
        imp = forest.best_estimator_.feature_importances_
        for k, col in enumerate(sampling_train['{}_X_{}'.format(j, i)].columns):
            f_imp_dic['{}_{}'.format(j, i)].at[k, 'column'] = col
            f_imp_dic['{}_{}'.format(j, i)].at[k, 'importance'] = imp[k]
        f_imp_dic['{}_{}'.format(j, i)] = f_imp_dic['{}_{}'.format(j, i)].sort_values('importance', ascending = False)
        count += 1

In [ ]:
for j in ['ros', 'rus', 'tl', 'smote', 'smt', 'bsmote', 'adasyn', 'cc']:#, 'smotenc']:
    for i in range(4, 5):
        i, j
        f_imp_dic['{}_{}'.format(j, i)].head(-1)

## Tomeklinks + Oversamplingを推定

In [ ]:
tl_resample_forest = pd.DataFrame(index = range(45),columns = ['サンプリング方法', 'データ番号', 'test auc', 'test accuracy', 'train auc', 'train accuracy', 'auc diff', 'best param'])
count = 0
f_imp_dic = {}
grid_param = {'max_depth': [5], 'min_samples_split': [2], 'min_samples_leaf': [2], 'n_estimators': [500]}
for j in ['ros', 'bsmote', 'adasyn']:#, 'smotenc']:
    for i in range(5):
        forest = GridSearchCV(RandomForestClassifier(random_state = 0), grid_param, cv = 5, scoring = 'roc_auc') 
        if j == 'tl':
            forest = GridSearchCV(RandomForestClassifier(random_state = 0, class_weight = 'balanced'), grid_param, cv = 5, scoring = 'roc_auc') 
        forest.fit(tl_over['{}_X_{}'.format(j, i)], tl_over['{}_y_{}'.format(j, i)])
        pred = forest.best_estimator_.predict(test[i])
        fpr, tpr, thresholds = roc_curve(y_test3, pred, pos_label = 1)
        test_auc = auc(fpr, tpr)
        tl_resample_forest['サンプリング方法'][count] = j
        tl_resample_forest['データ番号'][count] = i
        tl_resample_forest['best param'][count] = forest.best_params_
        tl_resample_forest['test auc'][count] = test_auc
        tl_resample_forest['test accuracy'][count] = accuracy_score(y_test3, pred) * 100
        pred = forest.best_estimator_.predict(tl_over['{}_X_{}'.format(j, i)])
        fpr, tpr, thresholds = roc_curve(tl_over['{}_y_{}'.format(j, i)], pred, pos_label = 1)
        train_auc = auc(fpr, tpr)
        tl_resample_forest['train auc'][count] = train_auc
        tl_resample_forest['train accuracy'][count] = accuracy_score(tl_over['{}_y_{}'.format(j, i)], pred) * 100
        tl_resample_forest['auc diff'][count] = train_auc - test_auc
        f_imp_dic['{}_{}'.format(j, i)] = pd.DataFrame(index = range(len(sampling_train['{}_X_{}'.format(j, i)].columns)))
        imp = forest.best_estimator_.feature_importances_
        for k, col in enumerate(sampling_train['{}_X_{}'.format(j, i)].columns):
            f_imp_dic['{}_{}'.format(j, i)].at[k, 'column'] = col
            f_imp_dic['{}_{}'.format(j, i)].at[k, 'importance'] = imp[k]
        f_imp_dic['{}_{}'.format(j, i)] = f_imp_dic['{}_{}'.format(j, i)].sort_values('importance', ascending = False)
        count += 1

In [ ]:
for j in ['ros', 'bsmote', 'adasyn']:#, 'smotenc']:
    for i in range(4, 5):
        i, j
        f_imp_dic['{}_{}'.format(j, i)].head(-1)

## 結果の出力

In [ ]:
#ライブラリそのまま使用
resample_forest = resample_forest.sort_values(['サンプリング方法', 'test auc'], ascending = False)
resample_forest.head(len(resample_forest))

In [ ]:
#Tomeklinks + Over sampling
tl_resample_forest = tl_resample_forest.sort_values(['サンプリング方法', 'test auc'], ascending = False)
tl_resample_forest.head(len(resample_forest))

In [ ]:
len(sampling_train['tl_y_3'].query('MAIS3 == 0'))
len(sampling_train['tl_y_4'].query('MAIS3 == 0'))

## 交差検定出力用リサンプリング
欠損値補完していないデータを入れる

In [ ]:
sampling_cv = {} #データ番号_cv番号_リサンプリング手法orテスト　を鍵にしてdfを格納
cv = 5
for i in range(5):
    t_df = df_dic[i].query('MAIS3 == 1').drop(columns = ['YEAR', 'MAIS'])
    t_len = len(t_df)
    t_cv = t_len // cv
    t_mod = t_len % cv
    f_df = df_dic[i].query('MAIS3 == 0').drop(columns = ['YEAR', 'MAIS'])
    f_len = len(f_df)
    f_cv = f_len // cv
    f_mod = f_len % cv
    for j in range(cv):
        if t_mod != 0:
            t_cv_df = t_df.sample(n = t_cv + 1, random_state = 0)
            t_mod -= 1
        else:
            t_cv_df = t_df.sample(n = t_cv, random_state = 0)
        t_df = t_df.drop(t_cv_df.index)
        if f_mod != 0:
            f_cv_df = f_df.sample(n = f_cv + 1, random_state = 0)
            f_mod -= 1
        else:
            f_cv_df = f_df.sample(n = f_cv, random_state = 0)
        f_df = f_df.drop(f_cv_df.index)
        sampling_cv['{}_{}_test'.format(i, j)] = pd.concat([t_cv_df, f_cv_df])
for i in range(5):
    for j in range(cv):
        sampling_cv['{}_{}_test_fillna'.format(i, j)] = sampling_cv['{}_{}_test'.format(i, j)]
        cv_list = list(range(cv))
        cv_list.remove(j)
        df = sampling_cv['{}_{}_test'.format(i, cv_list.pop(0))]
        for combine in cv_list:
            df = pd.concat([df, sampling_cv['{}_{}_test'.format(i, combine)]])
        for k in list(df.drop(columns = 'MAIS3').columns):
            med = df[k].median()
            df[k] = df[k].fillna(med)
            sampling_cv['{}_{}_test_fillna'.format(i, j)][k] = sampling_cv['{}_{}_test_fillna'.format(i, j)][k].fillna(med)
        trainy = df['MAIS3']
        trainX = df.drop(columns = ['MAIS3'])
        
        ros = RandomOverSampler(random_state = 0)
        bsmote = BorderlineSMOTE(random_state = 0)
        adasyn = ADASYN(random_state = 0)
        rus = RandomUnderSampler(random_state = 0)
        cc = ClusterCentroids(random_state = 0)
        smote = SMOTE(random_state = 0)
        smt = SMOTETomek(random_state = 0)
        tl = TomekLinks(random_state = 0)
        
        resample_list = [ros, bsmote, adasyn, rus, cc, smote, smt, tl, ros, bsmote, adasyn]
        resample_name = ['ros', 'bsmote', 'adasyn', 'rus', 'cc', 'smote', 'smt', 'tl', 'tlros', 'tlbsmote', 'tladasyn']
        for l, sampling_method in enumerate(resample_list):
            temp_df_X, temp_df_y = sampling_method.fit_sample(trainX, trainy.values.ravel())
            if resample_name[l] in ['ros', 'rus', 'tlros', 'tl']:
                temp_df_X, temp_df_y = pd.DataFrame(temp_df_X, columns = trainX.columns), pd.DataFrame(temp_df_y, columns = ['MAIS3'])
            else:
                temp_df_X, temp_df_y = pd.DataFrame(temp_df_X, columns = trainX.columns).round(), pd.DataFrame(temp_df_y, columns = ['MAIS3'])
            sampling_cv['{}_{}_{}'.format(i, j, resample_name[l])] = pd.concat([temp_df_X, temp_df_y], axis = 1)
            if resample_name[l] == 'tl':
                trainX, trainy = temp_df_X, temp_df_y

## リサンプリング結果を出力

In [ ]:
os.makedirs(os.path.join(path, 'Resampling_data'), exist_ok = True)
for i in range(5):
    for j in range(cv):
        sampling_cv['{}_{}_test_fillna'.format(i, j)] = sampling_cv['{}_{}_test_fillna'.format(i, j)].astype('int')
        sampling_cv['{}_{}_test_fillna'.format(i, j)].to_csv(os.path.join(path, 'Resampling_data', '{}_{}_test_fillna.csv'.format(i, j)), index = False)
        for k in ['ros', 'rus', 'cc', 'smote', 'smt', 'bsmote', 'adasyn', 'tlros', 'tlbsmote', 'tladasyn']:
            sampling_cv['{}_{}_{}'.format(i, j, k)] = sampling_cv['{}_{}_{}'.format(i, j, k)].astype('int')
            sampling_cv['{}_{}_{}'.format(i, j, k)].to_csv(os.path.join(path, 'Resampling_data', '{}_{}_{}.csv'.format(i, j, k)), index = False)

In [ ]:
os.makedirs(os.path.join(path, 'Resampling_data'), exist_ok = True)
for i in range(5):
    for j in range(cv):
        sampling_cv['{}_{}_test_fillna'.format(i, j)] = sampling_cv['{}_{}_test_fillna'.format(i, j)].astype('int')
        temp_df = sampling_cv['{}_{}_test_fillna'.format(i, j)].drop(columns = 'MAIS3')
        temp_df['MAIS3'] = np.nan
        temp_df.to_csv(os.path.join(path, 'Resampling_data', '{}_{}_test_bnet.csv'.format(i, j)), index = False)

## リサンプリング結果をcsvから読み込み

In [ ]:
sampling_method = ['ros', 'rus', 'cc', 'smote', 'smt', 'bsmote', 'adasyn', 'tlros', 'tlbsmote', 'tladasyn']
path = os.path.dirname(os.path.abspath('__file__'))
for i in range(5):
    for k in sampling_method:
        sampling_cv['{}_{}_test_fillna'.format(i, j)]  = pd.read_csv(os.path.join(path, 'Resampling_data', '{}_{}_test_fillna.csv'.format(i, j)))
        for j in range(cv):
               sampling_cv['{}_{}_{}'.format(i, j, k)] = pd.read_csv(os.path.join(path, 'Resampling_data', '{}_{}_{}.csv'.format(i, j, k)))

## ランダムフォレストで交差検定

In [ ]:
sampling_method = ['ros', 'rus', 'cc', 'smote', 'smt', 'bsmote', 'adasyn', 'tlros', 'tlbsmote', 'tladasyn']
resample_forest_cv = pd.DataFrame(index = range(5 * cv * len(sampling_method)),columns = ['サンプリング方法', 'データ番号', 'CV', 'MCC', 'PR-AUC',  'ROC-AUC', 'F', 'ACCURACY', 'RECALL', 'PRECISION'])
resample_forest_ave = pd.DataFrame(index = range(5 * len(sampling_method)),columns = ['サンプリング方法', 'データ番号', 'MCC', 'PR-AUC',  'ROC-AUC', 'F', 'ACCURACY', 'RECALL', 'PRECISION'])
f_imp_dic = {}
tree_data = {}
count = 0
count_ave = 0
for i in range(5):
    for k in sampling_method:
        for j in range(cv):
            #学習＆推定
            forest = RandomForestClassifier(random_state = 0, max_depth = 5, n_estimators = 500)
            forest.fit(sampling_cv['{}_{}_{}'.format(i, j, k)].drop(columns = 'MAIS3'), sampling_cv['{}_{}_{}'.format(i, j, k)]['MAIS3'])
            pred = forest.predict(sampling_cv['{}_{}_test_fillna'.format(i, j)].drop(columns = 'MAIS3'))
            pred_prob = forest.predict_proba(sampling_cv['{}_{}_test_fillna'.format(i, j)].drop(columns = 'MAIS3'))[:, 1]
            #精度
            y_true = sampling_cv['{}_{}_test_fillna'.format(i, j)]['MAIS3']
            mcc = matthews_corrcoef(y_true, pred)
            precision, recall, _ = precision_recall_curve(y_true, pred_prob)
            pr_auc = auc(recall, precision)
            accuracy = accuracy_score(y_true, pred)
            roc_auc = roc_auc_score(y_true, pred_prob)
            f_score = f1_score(y_true, pred, pos_label = 1)
            recall = recall_score(y_true, pred, pos_label = 1)
            precision = precision_score(y_true, pred, pos_label = 1)
            assign = [k, i, j, mcc, pr_auc, roc_auc, f_score, accuracy, recall, precision]
            for l, col in enumerate(['サンプリング方法', 'データ番号', 'CV', 'MCC', 'PR-AUC',  'ROC-AUC', 'F', 'ACCURACY', 'RECALL', 'PRECISION']):
                resample_forest_cv[col][count] = assign[l]
            #重要度
            f_imp_dic['{}_{}_{}'.format(i, j, k)] = pd.DataFrame(columns = ['column', 'importance'])
            imp = forest.feature_importances_
            for l, col in enumerate(sampling_cv['{}_{}_{}'.format(i, j,  k)].drop(columns = 'MAIS3').columns):
                f_imp_dic['{}_{}_{}'.format(i, j, k)].at[l, 'column'] = col
                f_imp_dic['{}_{}_{}'.format(i, j, k)].at[l, 'importance'] = imp[l]
            f_imp_dic['{}_{}_{}'.format(i, j, k)] = f_imp_dic['{}_{}_{}'.format(i, j, k)].sort_values('importance', ascending = False)
            #モデル保存
            tree_data['{}_{}_{}'.format(i, j, k)] = forest.estimators_
            count += 1
        #精度の平均を保存
        temp_df = resample_forest_cv.query('サンプリング方法 == "{}" & データ番号 == {}'.format(k, i))
        resample_forest_ave['サンプリング方法'][count_ave] = k
        resample_forest_ave['データ番号'][count_ave] = i
        for col in ['MCC', 'PR-AUC',  'ROC-AUC', 'F', 'ACCURACY', 'RECALL', 'PRECISION']:
            resample_forest_ave[col][count_ave] = temp_df[col].mean()
        #重要度の平均を保存
        df = f_imp_dic['{}_0_{}'.format(i, k)]
        for j in range(1, cv):
            df = pd.merge(df, f_imp_dic['{}_{}_{}'.format(i, j, k)], on = 'column', how = 'outer')
        for col in df.drop(columns = 'column').columns:
            df[col] = df[col].astype('float64')
        f_imp_dic['{}_{}_ave'.format(i, k)] = pd.concat([df['column'], df.sum(axis = 1).drop(columns = 'column')/5], axis = 1)
        count_ave += 1

## 結果の出力
平均  
- MCC
- PR-AUC
- F

In [ ]:
resample_forest_cv.head(len(resample_forest_cv))

In [ ]:
resample_forest_ave.sort_values('MCC', ascending = False)

In [ ]:
resample_forest_ave.sort_values('PR-AUC', ascending = False)

In [ ]:
resample_forest_ave.sort_values('F', ascending = False)

## 重要度の出力
cv毎の重要度を平均したものを出力

In [ ]:
for i in range(5):
    for j in sampling_method:
        i, j
        f_imp_dic['{}_{}_ave'.format(i, j)]

## 作成した決定木の出力
cv = 0の0, 5, -1番目の決定木を出力  

In [ ]:
sampling = 'ros'
data_num = 1
for i in [0, 5, -1]:
    tree_viz = dtreeviz(tree_data['{}_{}_{}'.format(data_num, 0, sampling)][i], 
                        sampling_cv['{}_{}_{}'.format(data_num, 0, sampling)].drop(columns = 'MAIS3'), 
                        sampling_cv['{}_{}_{}'.format(data_num, 0, sampling)]['MAIS3'], 
                        feature_names = sampling_cv['{}_{}_{}'.format(data_num, 0, sampling)].drop(columns = 'MAIS3').columns, 
                        target_name = 'MAIS3', 
                        class_names = [0, 1])
    #tree_viz.view()
    display(tree_viz)

## 使用した特徴量による精度比較
各リサンプリング手法毎に精度でソートし，各データが何位を取ったか数える  
例:ROSについて精度でソートするとデータ4が最も精度が高い  
       　→データ4の順位1に+1

In [ ]:
sort = 'ROC-AUC'
data_df =  pd.DataFrame(index = ['順位' + '{}'.format(i) for i in range(1, 6)],  columns = ['データ' + '{}'.format(i) for i in range(5)])
data_df.fillna(0, inplace = True)
for i in sampling_method:
    temp_df = resample_forest_ave.query('サンプリング方法 == "{}"'.format(i))
    temp_df = temp_df.sort_values(sort, ascending = False)
    for j in range(5):
        num = temp_df.iloc[j, 1]
        data_df.iloc[j, num] += 1
data_df

### 不均衡データにも使える3種の評価手法での合計
MCC, F値, PR-AUCを使用

In [ ]:
data_df =  pd.DataFrame(index = ['順位' + '{}'.format(i) for i in range(1, 6)],  columns = ['データ' + '{}'.format(i) for i in range(5)])
data_df.fillna(0, inplace = True)
for sort in ['F', 'MCC', 'PR-AUC']:
    for i in sampling_method:
        temp_df = resample_forest_ave.query('サンプリング方法 == "{}"'.format(i))
        temp_df = temp_df.sort_values(sort, ascending = False)
        for j in range(5):
            num = temp_df.iloc[j, 1]
            data_df.iloc[j, num] += 1
data_df

## リサンプリング手法の比較
データ0, データ2は精度が出ないので，データ1, 3, 4についてリサンプリング手法を比較する

In [ ]:
data_df =  pd.DataFrame(index = ['順位' + '{}'.format(i) for i in range(1, len(sampling_method) + 1)],  columns = sampling_method)
data_df.fillna(0, inplace = True)
for sort in ['F', 'MCC', 'PR-AUC']:
    for i in [1, 3, 4]:
        temp_df = resample_forest_ave.query('データ番号 == {}'.format(i))
        temp_df = temp_df.sort_values(sort, ascending = False)
        for j in range(len(sampling_method)):
            method_name = temp_df.iloc[j, 0]
            data_df[method_name][j] += 1
data_df

In [ ]:
data_df =  pd.DataFrame(index = ['順位' + '{}'.format(i) for i in range(1, len(sampling_method) + 1)],  columns = sampling_method)
data_df.fillna(0, inplace = True)
#for sort in ['F', 'MCC', 'PR-AUC']:
for sort in ['ROC-AUC']:
    for i in [1, 3, 4]:
        temp_df = resample_forest_ave.query('データ番号 == {}'.format(i))
        temp_df = temp_df.sort_values(sort, ascending = False)
        for j in range(len(sampling_method)):
            method_name = temp_df.iloc[j, 0]
            data_df[method_name][j] += 1
data_df

# ベイジアンネットワークでの推定結果の表示

## データの読み込み

In [ ]:
print('読み込みデータ(データ, リサンプリング方法)')
bayse_dic = {}
for i in range(5):
    for k in sampling_method:
        for j in range(cv):
            bayse_path = os.path.join(path, 'bayse_pred', '{}_{}'.format(i, k), '{}_{}_{}_pred.csv'.format(i, j, k)) 
            if os.path.exists(bayse_path):
                bayse_dic['{}_{}_{}'.format(i, j ,k)] = pd.read_csv(bayse_path)
        if os.path.exists(bayse_path):
            print('{}, {}'.format(i, k))
bayse_key = bayse_dic.keys()

## 評価

In [ ]:
cols = ['サンプリング方法', 'データ番号', 'CV', 'MCC', 'PR-AUC',  'ROC-AUC', 'F', 'ACCURACY', 'RECALL', 'PRECISION']
bayse_cv = pd.DataFrame(index = [], columns = cols)
cols.remove('CV')
bayse_ave = pd.DataFrame(index = [], columns = cols)
for i in range(5):
    for k in sampling_method:
        for j in range(cv):
            key = '{}_{}_{}'.format(i, j ,k)
            if key in bayse_key:
                pred = bayse_dic[key]['MAIS3']
                pred_prob = bayse_dic[key]['MAIS3.1']
                y_true = sampling_cv['{}_{}_test_fillna'.format(i, j)]['MAIS3']
                mcc = matthews_corrcoef(y_true, pred)
                precision, recall, _ = precision_recall_curve(y_true, pred_prob)
                pr_auc = auc(recall, precision)
                accuracy = accuracy_score(y_true, pred)
                roc_auc = roc_auc_score(y_true, pred_prob)
                f_score = f1_score(y_true, pred, pos_label = 1)
                recall = recall_score(y_true, pred, pos_label = 1)
                precision = precision_score(y_true, pred, pos_label = 1)
                record = pd.Series([k, i, j, mcc,  pr_auc, roc_auc, f_score, accuracy, recall, precision], index = bayse_cv.columns)
                bayse_cv = bayse_cv.append(record, ignore_index = True)
        if key in bayse_key:
            temp_df = bayse_cv.query('サンプリング方法 == "{}" & データ番号 == {}'.format(k, i))
            record = [k, i]
            for col in ['MCC', 'PR-AUC',  'ROC-AUC', 'F', 'ACCURACY', 'RECALL', 'PRECISION']:
                record.append(temp_df[col].mean())
            record = pd.Series(record, index = bayse_ave.columns)
            bayse_ave = bayse_ave.append(record, ignore_index = True)

## 結果の出力

In [ ]:
'MCC'
bayse_ave.sort_values('MCC', ascending = False).query('データ番号 == 3')
bayse_ave.sort_values('MCC', ascending = False).query('サンプリング方法 == "ros"')
'F'
bayse_ave.sort_values('F', ascending = False).query('データ番号 == 3')
bayse_ave.sort_values('F', ascending = False).query('サンプリング方法 == "ros"')
'PR-AUC'
bayse_ave.sort_values('PR-AUC', ascending = False).query('データ番号 == 3')
bayse_ave.sort_values('PR-AUC', ascending = False).query('サンプリング方法 == "ros"')
'ROC-AUC'
bayse_ave.sort_values('ROC-AUC', ascending = False).query('データ番号 == 3')
bayse_ave.sort_values('ROC-AUC', ascending = False).query('サンプリング方法 == "ros"')

# ロジスティック回帰で交差検定

## カテゴリ変数化

In [ ]:
col_name = ['MANCOLL', 'DVper10', 'BODYG', 'obodyg', 'BAGAVAIL',
                       'BAGAVOTH', 'MODELG', 'WGTG', 'owgtg', 'AGEG', 
                       'BMIGJP', 'PDOFG', 'MAIS3'] + ['SEATROW2', 'SEATLR2', 'SEX']
col_val = [list(range(x)) for x in [6, 11, 5, 6, 2, 2, 4, 3, 3, 3, 6, 4, 2]] + [list(range(1, 3)) for _ in range(3)]
for i in range(5):
    for j in range(cv):
        for k in sampling_method:
            for l, col in enumerate(col_name):
                if col in sampling_cv['{}_{}_{}'.format(i, j, k)].columns:
                    sampling_cv['{}_{}_{}'.format(i, j, k)][col] = pd.Categorical(sampling_cv['{}_{}_{}'.format(i, j, k)][col], categories = col_val[l])
            if col in sampling_cv['{}_{}_test_fillna'.format(i, j)].columns:
                sampling_cv['{}_{}_test_fillna'.format(i, j)][col] = pd.Categorical(sampling_cv['{}_{}_test_fillna'.format(i, j)][col], categories = col_val[l])

## ダミー変数化
説明変数の変更，カテゴリ変数の持つ値が変わると書き換える必要あり

In [ ]:
get_dummy = ['MANCOLL', 'BODYG', 'obodyg', 'PDOFG']
dummy_val = [list(range(6)), list(range(5)), list(range(6)), list(range(4))]
sampling_cv_dummy = {}
for i in range(5):
    for j in range(cv):
            for k in sampling_method:
                sampling_cv_dummy['{}_{}_{}'.format(i, j, k)] = pd.get_dummies(sampling_cv['{}_{}_{}'.format(i, j, k)], drop_first = True, columns = [x for x in get_dummy if x  in list(sampling_cv['{}_{}_{}'.format(i, j, k)].columns)])
            sampling_cv_dummy['{}_{}_test_fillna'.format(i, j)] = pd.get_dummies(sampling_cv['{}_{}_test_fillna'.format(i, j)], drop_first = True, columns = [x for x in get_dummy if x  in list(sampling_cv['{}_{}_test_fillna'.format(i, j)].columns)])

## 正規化

In [ ]:
for i in range(5):
    for j in range(cv):
        for k in sampling_method:
            mms = MinMaxScaler()
            mms.fit(sampling_cv_dummy['{}_{}_{}'.format(i, j, k)])
            col_order = sampling_cv_dummy['{}_{}_{}'.format(i, j, k)].columns
            sampling_cv_dummy['{}_{}_{}'.format(i, j, k)] = pd.DataFrame(mms.transform(sampling_cv_dummy['{}_{}_{}'.format(i, j, k)]), columns = col_order)
            sampling_cv_dummy['{}_{}_{}_test'.format(i, j, k)] = pd.DataFrame(mms.transform(sampling_cv_dummy['{}_{}_test_fillna'.format(i, j, k)][col_order]), columns = col_order)

## ロジスティック回帰で推定

In [ ]:
#logistic回帰パラメータ候補 C: 0.1, 0.5, 1
cols = ['サンプリング方法', 'データ番号', 'CV', 'MCC', 'PR-AUC',  'ROC-AUC', 'F', 'ACCURACY', 'RECALL', 'PRECISION']
logi_cv = pd.DataFrame(index = [], columns = cols)
cols.remove('CV')
logi_ave = pd.DataFrame(index = [], columns = cols)
logi_coef = {}
for i in range(5):
    for k in sampling_method:
        coef_df = pd.DataFrame(index = [], columns = ['CV'] + sampling_cv_dummy['{}_{}_{}'.format(i, j, k)].drop(columns = 'MAIS3').columns)
        i, k
        for j in range(cv):
            logi = LogisticRegression(random_state = 0, C = 1, solver = 'liblinear')
            logi.fit(sampling_cv_dummy['{}_{}_{}'.format(i, j, k)].drop(columns = 'MAIS3'), sampling_cv_dummy['{}_{}_{}'.format(i, j, k)]['MAIS3'])
            pred = logi.predict(sampling_cv_dummy['{}_{}_{}_test'.format(i, j, k)].drop(columns = 'MAIS3'))
            pred_prob = logi.predict_proba(sampling_cv_dummy['{}_{}_{}_test'.format(i, j, k)].drop(columns = 'MAIS3'))[:, 1]
            y_true = sampling_cv_dummy['{}_{}_{}_test'.format(i, j, k)]['MAIS3']
            #精度評価
            mcc = matthews_corrcoef(y_true, pred)
            precision, recall, _ = precision_recall_curve(y_true, pred_prob)
            pr_auc = auc(recall, precision)
            accuracy = accuracy_score(y_true, pred)
            roc_auc = roc_auc_score(y_true, pred_prob)
            f_score = f1_score(y_true, pred, pos_label = 1)
            recall = recall_score(y_true, pred, pos_label = 1)
            precision = precision_score(y_true, pred, pos_label = 1)
            record = pd.Series([k, i, j, mcc,  pr_auc, roc_auc, f_score, accuracy, recall, precision], index = logi_cv.columns)
            logi_cv = logi_cv.append(record, ignore_index = True)
            #係数
            record = pd.Series([j] + logi.coef_[0], index = ['CV'] + sampling_cv_dummy['{}_{}_{}'.format(i, j, k)].drop(columns = 'MAIS3').columns)
            coef_df = coef_df.append(record, ignore_index = True)
        logi_coef['{}_{}'.format(i, k)] = coef_df
        logi_coef['{}_{}_ave'.format(i, k)] = pd.DataFrame(coef_df.mean())
        #精度平均
        temp_df = logi_cv.query('サンプリング方法 == "{}" & データ番号 == {}'.format(k, i))
        record = [k, i]
        for col in ['MCC', 'PR-AUC',  'ROC-AUC', 'F', 'ACCURACY', 'RECALL', 'PRECISION']:
            record.append(temp_df[col].mean())
        record = pd.Series(record, index = bayse_ave.columns)
        logi_ave = logi_ave.append(record, ignore_index = True)

## 推定結果

In [ ]:
logi_ave.sort_values('F', ascending=False)

In [ ]:
for i in range(5):
    for k in sampling_method:
        i, k
        logi_coef['{}_{}_ave'.format(i, k)]

## リサンプリング(標準化後)

In [ ]:
sampling_train = {}
for i in range(5):
    trainsample = df_dic[i].query('YEAR < 2015')
    trainy = trainsample['MAIS3']
    trainX = trainsample.drop(columns = ['YEAR', 'MAIS', 'MAIS3'])
    #print('ros')
    ros = RandomOverSampler(random_state = 0)
    sampling_train['ros_X_{}'.format(i)], sampling_train['ros_y_{}'.format(i)] = ros.fit_sample(trainX, trainy)
    sampling_train['ros_X_{}'.format(i)] = pd.DataFrame(sampling_train['ros_X_{}'.format(i)], columns = [x for x in trainX.columns])
    sampling_train['ros_X_{}'.format(i)] = pd.get_dummies(sampling_train['ros_X_{}'.format(i)], drop_first = False, columns = [x for x in list(sampling_train['ros_X_{}'.format(i)].columns) if x not in not_dummy])
    #print('rus')
    rus = RandomUnderSampler(random_state = 0)
    sampling_train['rus_X_{}'.format(i)], sampling_train['rus_y_{}'.format(i)] = rus.fit_sample(trainX, trainy)
    sampling_train['rus_X_{}'.format(i)] = pd.DataFrame(sampling_train['rus_X_{}'.format(i)], columns = [x for x in trainX.columns])
    sampling_train['rus_X_{}'.format(i)] = pd.get_dummies(sampling_train['rus_X_{}'.format(i)], drop_first = False, columns = [x for x in list(sampling_train['rus_X_{}'.format(i)].columns) if x not in not_dummy])
    #print('tl')
    tl = TomekLinks(random_state = 0, ratio = 'majority')
    sampling_train['tl_X_{}'.format(i)], sampling_train['tl_y_{}'.format(i)] = tl.fit_sample(trainX, trainy)
    sampling_train['tl_X_{}'.format(i)] = pd.DataFrame(sampling_train['tl_X_{}'.format(i)], columns = [x for x in trainX.columns])
    sampling_train['tl_X_{}'.format(i)] = pd.get_dummies(sampling_train['tl_X_{}'.format(i)], drop_first = False, columns = [x for x in list(sampling_train['tl_X_{}'.format(i)].columns) if x not in not_dummy])
    #print('cc')
    cc = ClusterCentroids(random_state = 0, ratio = 'majority')
    sampling_train['cc_X_{}'.format(i)], sampling_train['cc_y_{}'.format(i)] = cc.fit_sample(trainX, trainy)
    sampling_train['cc_X_{}'.format(i)] = pd.DataFrame(sampling_train['cc_X_{}'.format(i)], columns = [x for x in trainX.columns])
    sampling_train['cc_X_{}'.format(i)] = sampling_train['cc_X_{}'.format(i)][[x for x in sampling_train['cc_X_{}'.format(i)].columns if x not in stdcol]].round() 
    sampling_train['cc_X_{}'.format(i)] = pd.get_dummies(sampling_train['cc_X_{}'.format(i)], drop_first = False, columns = [x for x in list(sampling_train['cc_X_{}'.format(i)].columns) if x not in not_dummy])
    #print('smote')
    smote = SMOTE(random_state = 0, ratio = 'minority')
    temp_df_X,sampling_train['smote_y_{}'.format(i)] = smote.fit_sample(trainX, trainy)
    temp_df_X = pd.DataFrame(temp_df_X, columns = [x for x in trainX.columns])
    sampling_train['smote_X_{}'.format(i)] = temp_df_X[[x for x in temp_df_X.columns if x not in stdcol]].round() 
    sampling_train['smote_X_{}'.format(i)] = pd.get_dummies(sampling_train['smote_X_{}'.format(i)], drop_first = False, columns = [x for x in list(sampling_train['smote_X_{}'.format(i)].columns) if x not in not_dummy])
    #print('smt')
    smt = SMOTETomek(random_state = 0, ratio = 'auto')
    temp_df_X, sampling_train['smt_y_{}'.format(i)] = smt.fit_sample(trainX, trainy)
    temp_df_X = pd.DataFrame(temp_df_X, columns = [x for x in trainX.columns])
    sampling_train['smt_X_{}'.format(i)] = temp_df_X[[x for x in temp_df_X.columns if x not in stdcol]].round() 
    sampling_train['smt_X_{}'.format(i)] = pd.get_dummies(sampling_train['smt_X_{}'.format(i)], drop_first = False, columns = [x for x in list(sampling_train['smt_X_{}'.format(i)].columns) if x not in not_dummy])

## SVMで推定(重い割にそこまで精度は出ない)

In [ ]:
svm_dic = {}
grid_param = {'C': [100, 500], 'gamma': [0.001, 0.01]}
for i in range(5):
    svm_dic[i] = GridSearchCV(svm.SVC(random_state = 0, class_weight = 'balanced'), grid_param, cv = 5, scoring = 'roc_auc')
    svm_dic[i].fit(train_dummy[i], y_train3)
    predict = svm_dic[i].predict(test_dummy[i])
    fpr, tpr, thresholds = roc_curve(y_test3, predict, pos_label = 1)
    print('best parameter:', svm_dic[i].best_params_)
    print('dic num:', i)
    print('test auc: ', auc(fpr, tpr))    
    predict = svm_dic[i].predict(train_dummy[i])
    fpr, tpr, thresholds = roc_curve(y_train3, predict, pos_label = 1)
    print('train auc: ', auc(fpr, tpr))